## Q11: Rock-Paper-Scissors

Implement a set of games of rock-paper-scissors against the computer.  

  * Ask for input from the user ("rock", "paper", or "scissors") and the randomly select one of these for the computer's play.
  * Announce who won.
  * Keep playing until a player says that they no longer want to play.
  * When all games are done, print out how many games were won by the player and by the computer 

In [ ]:
import random

class GameRPS:
    def __init__(self):
        self.playing = False
        self.computer_choices = ['rock','paper','scissors']
        self.current_user_choice = None
        self.wins = 0
        self.ties = 0
        self.loss = 0
        self.number_of_games = 0
        self.funny_map = {
            'rock': '🪨',
            'paper': '📄',
            'scissors': '✂️'
        }
    
    def communicate_with_user(self):
        if self.number_of_games == 0:
            print('Welcome to Rock 🪨, Paper 📄, Scissors ✂️!')
            print('Please type one of the following:')
            print('rock, paper, scissors')
        else:
            print('Do you want to play again?')
            print('Current score:')
            self.print_stats()
        print('You can also type "exit" to quit the game')
        user_input = input()
        if user_input in self.computer_choices:
            self.current_user_choice = user_input
            return True
        elif user_input == 'exit':
            return False
        else:
            print('Invalid choice, please try again')
            print('Please choose one of the following:')
            print('\t- rock')
            print('\t- paper')
            print('\t- scissors')
            print('\t- exit')
            self.communicate_with_user()
    
    def comparator(self,a,b):
        if a == b:
            return 0
        elif a == 'rock':
            if b == 'scissors':
                return 1
            else:
                return -1
        elif a == 'scissors':
            if b == 'paper':
                return 1
            else:
                return -1
        elif a == 'paper':
            if b == 'rock':
                return 1
            else:
                return -1
    
    def play(self):
        self.playing = True
        if(self.communicate_with_user()):
            computer_choice = random.choice(self.computer_choices)
            result = self.comparator(self.current_user_choice, computer_choice)
            print(f'You chose {self.current_user_choice} {self.funny_map[self.current_user_choice]}')
            print(f'The computer chose {computer_choice} {self.funny_map[computer_choice]}')
            self.number_of_games += 1
            self.wins += 1 if result == 1 else 0
            self.ties += 1 if result == 0 else 0
            self.loss += 1 if result == -1 else 0
            if result == 1:
                print('You won! 🎉')
            elif result == 0:
                print('It is a tie! 🤝')
            else:
                print('You lost! 😞')
            self.play()
        else:
            self.current_user_choice = None
            self.playing = False
            self.print_stats()
            self.goodbye()
    
    def goodbye(self):
        print('Thank you for playing, goodbye 👻!')
    
    def print_stats(self):
        print(f'Your score is {self.wins}/{self.number_of_games} with {self.ties} ties')
            
game = GameRPS()
game.play()

#if you want you can save the game (including the current stats) to pickle and restart playing
#import pickle
#with open('game.pickle', 'wb') as f:
#    pickle.dump(game, f)
#with open('game.pickle', 'rb') as f:
#    game = pickle.load(f)
#game.play()

## Q12: Pascal's triangle

Pascal's triangle is created such that each layer has 1 more element than the previous, with `1`s on the side and in the interior, the numbers are the sum of the two above it, e.g.,:
```
            1
          1   1
        1   2   1
      1   3   3   1
    1   4   6   4   1
  1   5   10  10  5   1
```

1. Write a function to return the first `n` rows of Pascal's triangle.  The return should be a list of length `n`, with each element itself a list containing the numbers for that row.
2. Write a function to print out Pascal's triangle with proper formatting, so the numbers in each row are centered between the ones in the row above

In [3]:
import numpy as np
import scipy as sp # to check accuracy of our brute force implementation

def ComputePascalTriangle(n_rows):
    if n_rows == 0:
        return []
    elif n_rows == 1:
        return [[1]]
    elif n_rows < 0:
        raise ValueError('The number of rows must be a positive integer')
    triangle = [[1],[1, 1]]
    def ComputeNextRow(triangle):
        last_row = triangle[-1]
        new_row = [0, *last_row] # I have noticed that if you add padding zeros and duplicate the last row, you can add them element-wise to get the next row
        last_row.append(0)
        new_row = np.add(new_row, last_row, dtype=np.object_) # Add element-wise with arbitrary precision
        last_row.pop()
        triangle.append(list(new_row))
        
    while len(triangle) < n_rows:
        ComputeNextRow(triangle)
    
    return triangle


def PrintFormattedPascalTriangle(triangle):
    # Assuming the triangle is computed correctly, it is guaranteed that the last line is the longest one
    if len(triangle)<12:
        def convert_triangle_to_string(triangle):
            sring_triangle = []
            for line in triangle:
                sring_triangle.append('   '.join(map(str, line)))
            return sring_triangle
        
        sring_triangle = convert_triangle_to_string(triangle)
        last_line_string = sring_triangle[-1]
        max_length = len(last_line_string) # Get the length of the last line as a string of single characters
        for line in sring_triangle:
            print(f'{line:^{max_length}}') #use f-string formatting to center with fixed width of the maximum line
    else:
        print("CANNOT PRINT ABOVE 12 LINES: above a certain line it is better to just get the coefficient you want as printing the whole triangle is not useful anymore. You can use either print(triangle[n][k] for the kth element in the nth row. For enhanced precision use sp.special.comb(n, k, exact=True) )")



def CheckPrecision(triangle, maximum_relative_error=1e-14, verbose = True):
    #above a certain line the relative error could increase if i don't use np.add(new_row, last_row, dtype=np.object_) because np.add overflows. Check that that function is working properly
    MAXIMUM_RELATIVE_ERROR = maximum_relative_error
    number_of_non_compliant = 0
    for i, line in enumerate(triangle):
        for j, element in enumerate(line):
            relative_error = abs((element - sp.special.comb(i, j, exact=True))/sp.special.comb(i, j, exact=True))
            if relative_error > MAXIMUM_RELATIVE_ERROR:
                if verbose:
                    print(f'Error at line {i} and element {j} \t {element} != {sp.special.comb(i, j, exact=True)} is {relative_error}')
                number_of_non_compliant += 1
    if number_of_non_compliant == 0:
        print(f'All elements are computed with the desired relative precision of {maximum_relative_error}. Notice that there could be absolute errors arbitrary large')
    else: 
        print(f'There are {number_of_non_compliant} elements that are not computed with the desired relative precision of {maximum_relative_error}')


triangle = ComputePascalTriangle(100)

CheckPrecision(triangle, maximum_relative_error=1e-14, verbose=True)
CheckPrecision(triangle, maximum_relative_error=1e-15, verbose=False)
CheckPrecision(triangle, maximum_relative_error=1e-16, verbose=False)
CheckPrecision(triangle, maximum_relative_error=1e-17, verbose=False)

PrintFormattedPascalTriangle(triangle)

triangle = ComputePascalTriangle(11)

CheckPrecision(triangle, maximum_relative_error=1e-14)
CheckPrecision(triangle, maximum_relative_error=1e-15)
CheckPrecision(triangle, maximum_relative_error=1e-16)
CheckPrecision(triangle, maximum_relative_error=1e-17)

PrintFormattedPascalTriangle(triangle)


All elements are computed with the desired relative precision of 1e-14. Notice that there could be absolute errors arbitrary large
All elements are computed with the desired relative precision of 1e-15. Notice that there could be absolute errors arbitrary large
There are 696 elements that are not computed with the desired relative precision of 1e-16
There are 696 elements that are not computed with the desired relative precision of 1e-17
CANNOT PRINT ABOVE 12 LINES: above a certain line it is better to just get the coefficient you want as printing the whole triangle is not useful anymore. You can use either print(triangle[n][k] for the kth element in the nth row. For enhanced precision use sp.special.comb(n, k, exact=True) )
All elements are computed with the desired relative precision of 1e-14. Notice that there could be absolute errors arbitrary large
All elements are computed with the desired relative precision of 1e-15. Notice that there could be absolute errors arbitrary large
All